# Mapping Myocardial Surface into Bull's eye plot

 - Extract centerline from base to apex
 - Extract cross-sections along the centerline
 - Rotate the coordinate system so that z-axis would be the centerline of the myocardium:
    - $phi = np.arccos(np.clip(np.dot(dir_centerline, [0, 0, 1]), -1, 1))$
    - $rotation matrix = [$

        $cos(phi),   sin(phi),   0$

        $-sin(phi),  cos(phi),   0$

        $0, 0, 1$
    
    $]$
 - on each cross section calculate and return:
    - $r = sqrt(x^2, y^2)$
    - $theta = arctan2(y, x)$
 - Reconstrcut final plot using r and theta, and return Ischemic and territory profile at each point.

In [1]:
import os

InputFolder = "/Users/ana/Documents/AnahitaSeresti/04_Tesselation/BullsEye/SU04A"
slice_base = os.path.join(InputFolder, "SliceBase.vtp")
slice_apex = os.path.join(InputFolder, "SliceApex.vtp")
myocardium = os.path.join(InputFolder, "MyocardiumSurface.vtp")
vesselterritory = os.path.join(InputFolder, "Projected_Territories_L_LCx_0+L_LCx_1+L_LCx_2.vtp")
ischemic = os.path.join(InputFolder, "Ischemic.vtp")



In [87]:
import glob

Files = glob.glob(f"{InputFolder}/*")
print(Files)

['/Users/ana/Documents/AnahitaSeresti/04_Tesselation/BullsEye/SU04A/LCx_0_map.vtp', '/Users/ana/Documents/AnahitaSeresti/04_Tesselation/BullsEye/SU04A/Map_R_PL_0.vtp', '/Users/ana/Documents/AnahitaSeresti/04_Tesselation/BullsEye/SU04A/centerline.vtp', '/Users/ana/Documents/AnahitaSeresti/04_Tesselation/BullsEye/SU04A/Map_R_PL_0_.vtp', '/Users/ana/Documents/AnahitaSeresti/04_Tesselation/BullsEye/SU04A/slice_test.vtp', '/Users/ana/Documents/AnahitaSeresti/04_Tesselation/BullsEye/SU04A/Map_L_Diag1_0.vtp', '/Users/ana/Documents/AnahitaSeresti/04_Tesselation/BullsEye/SU04A/slices_map_ischemia.vtp', '/Users/ana/Documents/AnahitaSeresti/04_Tesselation/BullsEye/SU04A/Projected_Territories_L_LCx_0+L_LCx_1+L_LCx_2.vtp', '/Users/ana/Documents/AnahitaSeresti/04_Tesselation/BullsEye/SU04A/slices_map.vtp', '/Users/ana/Documents/AnahitaSeresti/04_Tesselation/BullsEye/SU04A/Map_aorta.vtp', '/Users/ana/Documents/AnahitaSeresti/04_Tesselation/BullsEye/SU04A/slices_map_ischemic.vtp', '/Users/ana/Document

In [2]:
import vtk
import numpy as np
from utilities import ReadVTPFile, GetCentroid, WriteVTPFile

centeroid_base = GetCentroid(ReadVTPFile(slice_base))
centeroid_apex = GetCentroid(ReadVTPFile(slice_apex))
Myocardium = ReadVTPFile(myocardium)
Ischemic = ReadVTPFile(ischemic)
VesselTerritory = ReadVTPFile(vesselterritory)

In [94]:
Myocardium.GetPointData().GetNumberOfArrays()
Myocardium.GetPointData().GetArrayName(2)

for i in range(Myocardium.GetPointData().GetNumberOfArrays()):
    array_name_ = Myocardium.GetPointData().GetArrayName(i)
    if "Ischemic" in array_name_:
        IschemicArrayName = array_name_
    elif "TerritoryLabels" in array_name_:
        TerritoryArrayName = "SelectedTerritory"
    else:
        continue

print(IschemicArrayName)
print(TerritoryArrayName)

IschemicProfile
SelectedTerritory


In [3]:
def Line(point1, point2, res):
    line = vtk.vtkLineSource()
    line.SetPoint1(point1)
    line.SetPoint2(point2)
    line.SetResolution(res)
    line.Update()
    
    return line.GetOutput()

def BoldLine(centerline):
    tube_filter = vtk.vtkTubeFilter()
    tube_filter.SetInputData(centerline)
    tube_filter.SetRadius(0.05)  # Adjust this value to change thickness
    tube_filter.SetNumberOfSides(50)  # Higher = smoother tube
    tube_filter.CappingOn()  # Close tube ends
    tube_filter.Update()
    
    return tube_filter.GetOutput()

def ThresholdInBetweenPoints(Surface, arrayname, value1, value2):
    Threshold=vtk.vtkThresholdPoints()
    Threshold.SetInputData(Surface)
    Threshold.SetLowerThreshold(value1)
    Threshold.SetUpperThreshold(value2)
    #Threshold.ThresholdBetween(value1,value2)
    Threshold.SetInputArrayToProcess(0,0,0,vtk.vtkDataObject.FIELD_ASSOCIATION_POINTS,arrayname)
    Threshold.Update()
    return Threshold.GetOutput()

In [17]:
CL_dir = np.array([
    centeroid_base[0] - centeroid_apex[0],
    centeroid_base[1] - centeroid_apex[1],
    centeroid_base[2] - centeroid_apex[2],
])
CL_direction = CL_dir/np.linalg.norm(CL_dir)

point0 = np.array([
    centeroid_base[0] + CL_direction[0]*5,
    centeroid_base[1] + CL_direction[1]*5,
    centeroid_base[2] + CL_direction[2]*5
])

point1 = np.array([
    centeroid_apex[0] - CL_direction[0]*1.5,
    centeroid_apex[1] - CL_direction[1]*1.5,
    centeroid_apex[2] - CL_direction[2]*1.5
])

centerline = Line(point0,point1,1000)
WriteVTPFile(os.path.join(InputFolder, "centerline.vtp"), BoldLine(centerline))

In [18]:
def SliceWPlane(Volume,Origin,Norm):
    plane=vtk.vtkPlane()
    plane.SetOrigin(Origin)
    plane.SetNormal(Norm)
    Slice=vtk.vtkCutter()
    Slice.GenerateTrianglesOff()
    Slice.SetCutFunction(plane)
    Slice.SetInputData(Volume)
    Slice.Update()
    
    return Slice.GetOutput()

from scipy.spatial.transform import Rotation as R
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk

append_filter = vtk.vtkAppendPolyData()
for i in range(centerline.GetNumberOfPoints()):
    point = centerline.GetPoint(i)
    slice_ = SliceWPlane(Myocardium, point, CL_direction)
    
    append_filter.AddInputData(slice_)

append_filter.Update()
WriteVTPFile(os.path.join(InputFolder, "slices.vtp"), append_filter.GetOutput())


In [19]:
#from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk
from scipy.spatial.transform import Rotation as R
i = 20
point = centerline.GetPoint(i)
slice_ = SliceWPlane(Myocardium, point, CL_direction)
IschemicProfile = slice_.GetPointData().GetArray("IschemicProfile")
coords = []
rotation, _ = R.align_vectors([[0, 0, 1]], [CL_direction])
r_ = 0
theta_ = []
for i in range(slice_.GetNumberOfPoints()):
    coord_ = slice_.GetPoint(i)
    aligned_coord_ = rotation.apply(coord_)
    theta_.append(np.arctan2(aligned_coord_[1], aligned_coord_[0]))
    r_ += np.sqrt(aligned_coord_[0]**2 + aligned_coord_[1]**2)
    coords.append(aligned_coord_)

r_ /= slice_.GetNumberOfPoints()
print(r_)
new_coords = []
for angle in theta_:
    new_coords.append([r_*np.cos(angle), r_*np.sin(angle), 0])

new_points = vtk.vtkPoints()
for pt in new_coords:
    new_points.InsertNextPoint(pt)

new_polydata = vtk.vtkPolyData()
new_polydata.SetPoints(new_points)

ProfileCopy = vtk.vtkFloatArray()
ProfileCopy.SetName("IschemicProfile")
ProfileCopy.SetNumberOfComponents(1)
ProfileCopy.SetNumberOfTuples(slice_.GetNumberOfPoints())

for i in range(slice_.GetNumberOfPoints()):
    ProfileCopy.SetTuple(i, IschemicProfile.GetTuple(i))

new_polydata.GetPointData().AddArray(ProfileCopy)

cells = vtk.vtkCellArray()
n_pts = new_points.GetNumberOfPoints()

polygon = vtk.vtkPolygon()
polygon.GetPointIds().SetNumberOfIds(n_pts + 1)  # +1 to close the loop

for idx in range(n_pts):
    polygon.GetPointIds().SetId(idx, idx)
polygon.GetPointIds().SetId(n_pts, 0)  # close loop

cells.InsertNextCell(polygon)
#new_polydata.SetLines(cells)
new_polydata.SetPolys(cells)

WriteVTPFile(os.path.join(InputFolder, "slice_test.vtp"), new_polydata)



/var/folders/05/bygg4rdj3n7543jkngl6k6z40000gn/T/ipykernel_14047/1478551897.py:8: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  rotation, _ = R.align_vectors([[0, 0, 1]], [CL_direction])


ZeroDivisionError: division by zero

In [46]:
from scipy.spatial.transform import Rotation as R
from utilities import ThresholdPointsByUpper
rotation, _ = R.align_vectors([[0, 0, 1]], [CL_direction])
append_filter = vtk.vtkAppendPolyData()
append_filter2 = vtk.vtkAppendPolyData()
Radii = []
for i in range(centerline.GetNumberOfPoints()):
    point = centerline.GetPoint(i)
    slice_ = SliceWPlane(Myocardium, point, CL_direction)
    if slice_.GetNumberOfPoints() == 0: 
        continue

    IschemicProfile = slice_.GetPointData().GetArray("IschemicProfile")
    coords = []
    r_ = 0
    for j in range(slice_.GetNumberOfPoints()):
        coord_ = slice_.GetPoint(j)
        aligned_coord_ = rotation.apply(coord_)
        r_ += np.sqrt(aligned_coord_[0]**2 + aligned_coord_[1]**2)
        coords.append(coord_)

    r_ /= slice_.GetNumberOfPoints()
    Radii.append(r_)

R_max = max(Radii)
R_map = [i for i in range(centerline.GetNumberOfPoints(), 0, -1) * R_max/centerline.GetNumberOfPoints()]

SurfaceArea_I = []
Center = []
for i in range(centerline.GetNumberOfPoints()):
    point = centerline.GetPoint(i)
    slice_ = SliceWPlane(Myocardium, point, CL_direction)
    if slice_.GetNumberOfPoints() == 0:
        Center.append(point)
        continue

    SurfaceArea_M = slice_.GetNumberOfPoints()
    IschemicProfile = slice_.GetPointData().GetArray("IschemicProfile")
    Ischemic_Threshold = ThresholdPointsByUpper(slice_, "IschemicProfile", 1)
    SurfaceArea_I.append(Ischemic_Threshold.GetNumberOfPoints()/SurfaceArea_M)

    coords = []
    r_ = 0
    theta_ = []
    pts_np = np.array([slice_.GetPoint(j) for j in range(slice_.GetNumberOfPoints())])
    center = pts_np.mean(axis=0)
    Center.append(center)
    for j in range(slice_.GetNumberOfPoints()):
        coord_ = slice_.GetPoint(j)
        aligned_coord_ = rotation.apply(coord_ - center)
        theta_.append(np.arctan2(aligned_coord_[1], aligned_coord_[0]))
        coords.append(coord_)

    new_coords = []
    for angle in theta_:
        new_coords.append([R_map[i]*np.cos(angle), R_map[i]*np.sin(angle), 0])
    
    new_points = vtk.vtkPoints()
    for pt in new_coords:
        new_points.InsertNextPoint(pt)

    new_polydata = vtk.vtkPolyData()
    new_polydata.SetPoints(new_points)

    ProfileCopy = vtk.vtkFloatArray()
    ProfileCopy.SetName("IschemicProfile")
    ProfileCopy.SetNumberOfComponents(1)
    ProfileCopy.SetNumberOfTuples(slice_.GetNumberOfPoints())

    for j in range(slice_.GetNumberOfPoints()):
        ProfileCopy.SetTuple(j, IschemicProfile.GetTuple(j))

    new_polydata.GetPointData().AddArray(ProfileCopy)
    """cells = vtk.vtkCellArray()
    n_pts = new_points.GetNumberOfPoints()

    polygon = vtk.vtkPolygon()
    polygon.GetPointIds().SetNumberOfIds(n_pts + 1)  # +1 to close the loop

    for idx in range(n_pts):
        polygon.GetPointIds().SetId(idx, idx)
    polygon.GetPointIds().SetId(n_pts, 0)  # close loop

    cells.InsertNextCell(polygon)
    new_polydata.SetPolys(cells)"""

    append_filter.AddInputData(new_polydata)

    # Territory Assignement

    TerritoryProfile = slice_.GetPointData().GetArray(2)
    
    new_points = vtk.vtkPoints()
    for pt in new_coords:
        new_points.InsertNextPoint(pt)

    new_polydata = vtk.vtkPolyData()
    new_polydata.SetPoints(new_points)

    ProfileCopy = vtk.vtkFloatArray()
    ProfileCopy.SetName("SelectedTerritory")
    ProfileCopy.SetNumberOfComponents(1)
    ProfileCopy.SetNumberOfTuples(slice_.GetNumberOfPoints())

    for j in range(slice_.GetNumberOfPoints()):
        ProfileCopy.SetTuple(j, TerritoryProfile.GetTuple(j))

    new_polydata.GetPointData().AddArray(ProfileCopy)
    """cells = vtk.vtkCellArray()
    n_pts = new_points.GetNumberOfPoints()

    polygon = vtk.vtkPolygon()
    polygon.GetPointIds().SetNumberOfIds(n_pts + 1)  # +1 to close the loop

    for idx in range(n_pts):
        polygon.GetPointIds().SetId(idx, idx)
    polygon.GetPointIds().SetId(n_pts, 0)  # close loop

    cells.InsertNextCell(polygon)
    new_polydata.SetPolys(cells)"""

    append_filter2.AddInputData(new_polydata)


append_filter.Update()
append_filter2.Update()

delaunay = vtk.vtkDelaunay2D()
delaunay.SetInputData(append_filter.GetOutput())
delaunay.Update()
IschemicMap = delaunay.GetOutput()

delaunay2 = vtk.vtkDelaunay2D()
delaunay2.SetInputData(append_filter2.GetOutput())
delaunay2.Update()
TerritoryMap = delaunay2.GetOutput()


WriteVTPFile(os.path.join(InputFolder, "slices_map.vtp"), IschemicMap)
WriteVTPFile(os.path.join(InputFolder, "territory_map.vtp"), TerritoryMap)



/var/folders/05/bygg4rdj3n7543jkngl6k6z40000gn/T/ipykernel_14047/516182737.py:3: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  rotation, _ = R.align_vectors([[0, 0, 1]], [CL_direction])


In [21]:
def ConvertPointsToPolyData(points):
    polydata = vtk.vtkPolyData()
    polydata.SetPoints(points)

    delaunay = vtk.vtkDelaunay2D()
    delaunay.SetInputData(polydata)
    delaunay.Update()

    return delaunay.GetOutput()


In [84]:
"""from scipy.spatial.transform import Rotation as R
from utilities import ThresholdPointsByUpper
rotation, _ = R.align_vectors([[0, 0, 1]], [CL_direction])
append_filter = vtk.vtkAppendPolyData()
append_filter2 = vtk.vtkAppendPolyData()
Radii = []
for i in range(centerline.GetNumberOfPoints()):
    point = centerline.GetPoint(i)
    slice_ = SliceWPlane(Myocardium, point, CL_direction)
    if slice_.GetNumberOfPoints() == 0: 
        continue

    IschemicProfile = slice_.GetPointData().GetArray("IschemicProfile")
    coords = []
    r_ = 0
    for j in range(slice_.GetNumberOfPoints()):
        coord_ = slice_.GetPoint(j)
        aligned_coord_ = rotation.apply(coord_)
        r_ += np.sqrt(aligned_coord_[0]**2 + aligned_coord_[1]**2)
        coords.append(coord_)

    r_ /= slice_.GetNumberOfPoints()
    Radii.append(r_)

R_max = max(Radii)
R_map = [i for i in range(centerline.GetNumberOfPoints(), 0, -1) * R_max/centerline.GetNumberOfPoints()]

SurfaceArea_I = []
for i in range(centerline.GetNumberOfPoints()):
    point = centerline.GetPoint(i)
    slice_ = SliceWPlane(Myocardium, point, CL_direction)
    if slice_.GetNumberOfPoints() == 0: 
        continue

    SurfaceArea_M = slice_.GetNumberOfPoints()
    IschemicProfile = slice_.GetPointData().GetArray("IschemicProfile")
    Ischemic_Threshold = ThresholdPointsByUpper(slice_, "IschemicProfile", 1)
    SurfaceArea_I.append(Ischemic_Threshold.GetNumberOfPoints()/SurfaceArea_M)

    coords = []
    r_ = 0
    theta_ = []
    pts_np = np.array([slice_.GetPoint(j) for j in range(slice_.GetNumberOfPoints())])
    center = pts_np.mean(axis=0)
    for j in range(slice_.GetNumberOfPoints()):
        coord_ = slice_.GetPoint(j)
        aligned_coord_ = rotation.apply(coord_ - center)
        theta_.append(np.arctan2(aligned_coord_[1], aligned_coord_[0]))
        coords.append(coord_)

    new_coords = []
    for angle in theta_:
        new_coords.append([R_map[i]*np.cos(angle), R_map[i]*np.sin(angle), 0])
    
    new_points = vtk.vtkPoints()
    for pt in new_coords:
        new_points.InsertNextPoint(pt)

    new_polydata = ConvertPointsToPolyData(new_points)
    

    ProfileCopy = vtk.vtkFloatArray()
    ProfileCopy.SetName("IschemicProfile")
    ProfileCopy.SetNumberOfComponents(1)
    ProfileCopy.SetNumberOfTuples(slice_.GetNumberOfPoints())

    for j in range(slice_.GetNumberOfPoints()):
        ProfileCopy.SetTuple(j, IschemicProfile.GetTuple(j))

    new_polydata.GetPointData().AddArray(ProfileCopy)
    
    append_filter.AddInputData(new_polydata)

    # Territory Assignement

    TerritoryProfile = slice_.GetPointData().GetArray(2)
    
    new_points = vtk.vtkPoints()
    for pt in new_coords:
        new_points.InsertNextPoint(pt)

    new_polydata = ConvertPointsToPolyData(new_points)

    ProfileCopy = vtk.vtkFloatArray()
    ProfileCopy.SetName("SelectedTerritory")
    ProfileCopy.SetNumberOfComponents(1)
    ProfileCopy.SetNumberOfTuples(slice_.GetNumberOfPoints())

    for j in range(slice_.GetNumberOfPoints()):
        ProfileCopy.SetTuple(j, TerritoryProfile.GetTuple(j))

    new_polydata.GetPointData().AddArray(ProfileCopy)
    
    append_filter2.AddInputData(new_polydata)


append_filter.Update()
append_filter2.Update()

WriteVTPFile(os.path.join(InputFolder, "slices_map.vtp"), append_filter.GetOutput())
WriteVTPFile(os.path.join(InputFolder, "territory_map.vtp"), append_filter2.GetOutput())
"""
from utilities import ThresholdByUpper, ExtractSurface
AppendFilter = vtk.vtkAppendFilter()
AppendFilter.AddInputData(IschemicMap)
AppendFilter.Update()
IschemicRegionMap = ThresholdByUpper(AppendFilter.GetOutput(), "IschemicProfile", 1)

AppendFilter = vtk.vtkAppendFilter()
AppendFilter.AddInputData(TerritoryMap)
AppendFilter.Update()
VesselTerritoryMap = ThresholdByUpper(AppendFilter.GetOutput(), "SelectedTerritory", 1)

WriteVTPFile(os.path.join(InputFolder, "slices_map_ischemic.vtp"), ExtractSurface(IschemicRegionMap))
WriteVTPFile(os.path.join(InputFolder, "territory_map_territory.vtp"), ExtractSurface(VesselTerritoryMap))


2025-05-15 01:22:37.417 (29486.830s) [          2D5D57]       vtkThreshold.cxx:87    WARN| vtkThreshold::ThresholdByUpper was deprecated for VTK 9.1 and will be removed in a future version.
2025-05-15 01:22:37.596 (29487.009s) [          2D5D57]       vtkThreshold.cxx:87    WARN| vtkThreshold::ThresholdByUpper was deprecated for VTK 9.1 and will be removed in a future version.


In [96]:
R_max_ = 12.0
Npoints  = centerline.GetNumberOfPoints()
R_map = [i * R_max_/Npoints for i in range(Npoints, 0, -1)]
print(R_map)

[12.0, 11.988011988011989, 11.976023976023976, 11.964035964035965, 11.952047952047952, 11.94005994005994, 11.928071928071928, 11.916083916083917, 11.904095904095904, 11.892107892107893, 11.88011988011988, 11.868131868131869, 11.856143856143856, 11.844155844155845, 11.832167832167832, 11.82017982017982, 11.808191808191808, 11.796203796203796, 11.784215784215784, 11.772227772227772, 11.76023976023976, 11.748251748251748, 11.736263736263735, 11.724275724275724, 11.712287712287713, 11.7002997002997, 11.688311688311689, 11.676323676323676, 11.664335664335665, 11.652347652347652, 11.640359640359641, 11.628371628371628, 11.616383616383617, 11.604395604395604, 11.592407592407593, 11.58041958041958, 11.568431568431569, 11.556443556443556, 11.544455544455545, 11.532467532467532, 11.52047952047952, 11.508491508491508, 11.496503496503497, 11.484515484515484, 11.472527472527473, 11.46053946053946, 11.448551448551449, 11.436563436563437, 11.424575424575425, 11.412587412587413, 11.4005994005994, 11.3

In [23]:
CenteroidIschemic = GetCentroid(IschemicRegionMap)
CenteroidVesselTerritory = GetCentroid(VesselTerritoryMap)

Centeroid_CircularMap = [0, 0, 0]

Line1 = Line(Centeroid_CircularMap, CenteroidIschemic, 10)
Line2 = Line(Centeroid_CircularMap, CenteroidVesselTerritory, 10)

WriteVTPFile(os.path.join(InputFolder, "LineIschemicCenter.vtp"), BoldLine(Line1))
WriteVTPFile(os.path.join(InputFolder, "LineTerritoryCenter.vtp"), BoldLine(Line2))

direction1 = [CenteroidIschemic[0] - Centeroid_CircularMap[0],
              CenteroidIschemic[1] - Centeroid_CircularMap[1],
              CenteroidIschemic[2] - Centeroid_CircularMap[2]
              ]

direction1 /= np.linalg.norm(direction1)

direction2 = [CenteroidVesselTerritory[0] - Centeroid_CircularMap[0],
              CenteroidVesselTerritory[1] - Centeroid_CircularMap[1],
              CenteroidVesselTerritory[2] - Centeroid_CircularMap[2]
              ]

direction2 /= np.linalg.norm(direction1)

dot_product = np.clip(np.dot(direction1, direction2), -1.0, 1.0)
angle_rad = np.arccos(dot_product)
angle_degree = np.degrees(angle_rad)
print(angle_degree)

0.0


In [ ]:
from scipy.spatial.transform import Rotation as R
from utilities import LargestConnectedRegion
rotation, _ = R.align_vectors([[0, 0, 1]], [CL_direction])
append_filter = vtk.vtkAppendPolyData()


SurfaceArea_T = []
for i in range(centerline.GetNumberOfPoints()):
    point = centerline.GetPoint(i)
    slice_m = SliceWPlane(Myocardium, point, CL_direction)
    slice_ = SliceWPlane(Ischemic, point, CL_direction)
    if slice_.GetNumberOfPoints() == 0: 
        continue

    SurfaceArea_M2 = slice_m.GetNumberOfPoints()
    SurfaceArea_T.append(slice_.GetNumberOfPoints()/SurfaceArea_M2)
    #IschemicProfile = slice_.GetPointData().GetArray("IschemicProfile")
    coords = []
    r_ = 0
    theta_ = []
    pts_np = np.array([slice_.GetPoint(j) for j in range(slice_.GetNumberOfPoints())])
    center = pts_np.mean(axis=0)
    for j in range(slice_.GetNumberOfPoints()):
        coord_ = slice_.GetPoint(j)
        aligned_coord_ = rotation.apply(coord_ - center)
        theta_.append(np.arctan2(aligned_coord_[1], aligned_coord_[0]))
        coords.append(coord_)

    new_coords = []
    for angle in theta_:
        new_coords.append([R_map[i]*np.cos(angle), R_map[i]*np.sin(angle), 0])
    
    new_points = vtk.vtkPoints()
    for pt in new_coords:
        new_points.InsertNextPoint(pt)
    
    polydata = vtk.vtkPolyData()
    polydata.SetPoints(new_points)

    hull  = vtk.vtkConvexHull2D()
    hull.SetInputData(polydata)
    hull.Update()

    append_filter.AddInputData(hull.GetOutput())

append_filter.Update()
#triangulate = vtk.vtkTriangleFilter()
#triangulate.SetInputData(append_filter.GetOutput())
#triangulate.Update()
WriteVTPFile(os.path.join(InputFolder, "territory_map.vtp"), append_filter.GetOutput())

In [ ]:
import matplotlib.pyplot as plt

plt.plot(SurfaceArea_I, label = "Ischemic Region", color = "tomato")
plt.plot(SurfaceArea_T, label = "Vessel Territory", color = "olivedrab")
plt.ylabel("Surface Area of Region per Surface Area of Myocardium")
plt.xticks([])

x_min, x_max = plt.xlim()
lines_x = [x_min + (x_max - x_min) * frac for frac in [1/5, 1/2, 4/5]]
labels = ['Basal', 'Mid', 'Apical']

for i, x_pos in enumerate(lines_x):
    plt.axvline(x=x_pos, color='gray', linestyle='--')
    plt.text(x_pos , plt.ylim()[0], labels[i], rotation=90, va='top', ha='center')

plt.legend()
plt.tight_layout()
plt.show()

#

# Map Coronaries onto the Polar Plot

In [ ]:
def ReorderCoronary(new_points, original_cl):
    new_points = set(new_points)
    original_cl = np.asarray(original_cl)
    for point in new_points:
        point = np.array(point)
        differences = np.abs(original_cl - point)
        closest_idx = np.argmin(differences)

In [50]:
PathFolder = os.path.join(InputFolder, "Paths/*.vtp")
import glob

import sys
import argparse

TessellationPath = os.path.abspath(f"{sys.path[0]}/..")
sys.path.append(TessellationPath)

from SimVascularTools.ConvertPathtoVTP import ConvertPath2VTP

args = argparse.Namespace()
args.InputFolder = os.path.join(InputFolder, "Paths")
Path2Point = ConvertPath2VTP(args)
Pathfiles = glob.glob(PathFolder)
rotation, _ = R.align_vectors([[0, 0, 1]], [CL_direction])

#file_ = ReadVTPFile(PathFolder)
Npoints  = centerline.GetNumberOfPoints()
for path in Pathfiles:
    file_ = ReadVTPFile(path)
    new_points = []
    arbitrary_points = []
    for i in range(Npoints):
        point = centerline.GetPoint(i)
        slice_ = SliceWPlane(file_, point, CL_direction)
    
        if slice_.GetNumberOfPoints() == 0:
            continue
        for j in range(slice_.GetNumberOfPoints()):
            arbitrary_points.append(slice_.GetPoint(j))
            centered_point = [
                slice_.GetPoint(j)[0] - Center[i][0],
                slice_.GetPoint(j)[1] - Center[i][1],
                slice_.GetPoint(j)[2] - Center[i][2]
            ]
            alignPoints = rotation.apply(centered_point)
            theta_ = np.arctan2(alignPoints[1], alignPoints[0])
            new_points.append([R_map[i]*np.cos(theta_), R_map[i]*np.sin(theta_), 0])

    vessel_polydata = Path2Point.points_to_vtp(new_points)
    VesselName = os.path.splitext(os.path.basename(path))[0]
    print(f"--- Converting {VesselName}")
    Vesseldir = os.path.join(InputFolder, f"Map_{VesselName}.vtp")
    WriteVTPFile(Vesseldir, BoldLine(vessel_polydata))






/var/folders/05/bygg4rdj3n7543jkngl6k6z40000gn/T/ipykernel_14047/467459359.py:16: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  rotation, _ = R.align_vectors([[0, 0, 1]], [CL_direction])


--- Converting L_LCx_2
--- Converting L_LCx_3
--- Converting L_LCx_1
--- Converting No_Myo_0
--- Converting No_Myo_1
--- Converting L_LCx_0
--- Converting R_RCA_0
--- Converting L_LAD_2
--- Converting L_LAD_3
--- Converting L_LAD_1
--- Converting L_LAD_0
--- Converting aorta
--- Converting L_Diag2_0
--- Converting R_PL_0_
--- Converting R_PDA_0
--- Converting R_PL_0
--- Converting R_PDA_1
--- Converting L_Diag1_0


In [75]:

def FindClosestPoint(array, point):
    point = np.array(point)
    array = np.asarray(array)
    differences = np.linalg.norm(array - point, axis=1)
    idx = np.argmin(differences)
    
    return idx


point = (4.669581816693317, -17.5139976189785, -18.0537580163916)
point = np.array(point)
array = np.asarray(Center)
differences = np.linalg.norm(array - point, axis=1)
print(len(differences))
idx = np.argmin(differences)
print(idx)

1001
330


In [79]:
def ReorderArray(arbitrary_array, VesselCenterLine, new_points):
    idx_array = []
    for point in arbitrary_array:
        idx_array.append(FindClosestPoint(VesselCenterLine, point))
    sorted_idx = np.argsort(np.array(idx_array))
    new_points = np.array(new_points)
    return new_points[sorted_idx]

In [82]:
PathFolder = os.path.join(InputFolder, "Paths/*.vtp")
import glob

import sys
import argparse

TessellationPath = os.path.abspath(f"{sys.path[0]}/..")
sys.path.append(TessellationPath)

from SimVascularTools.ConvertPathtoVTP import ConvertPath2VTP

args = argparse.Namespace()
args.InputFolder = os.path.join(InputFolder, "Paths")
Path2Point = ConvertPath2VTP(args)
Pathfiles = glob.glob(PathFolder)
rotation, _ = R.align_vectors([[0, 0, 1]], [CL_direction])

#file_ = ReadVTPFile(PathFolder)
Npoints  = centerline.GetNumberOfPoints()
for path in Path2Point.filenames:
    VesselCenterline, _ = Path2Point.pth_to_points(path)
    file_ = Path2Point.points_to_vtp(VesselCenterline)
    new_points = []
    arbitrary_points = []
    for i in range(Npoints):
        point = centerline.GetPoint(i)
        slice_ = SliceWPlane(file_, point, CL_direction)
    
        if slice_.GetNumberOfPoints() == 0:
            continue
        for j in range(slice_.GetNumberOfPoints()):
            arbitrary_points.append(slice_.GetPoint(j))
            centered_point = [
                slice_.GetPoint(j)[0] - Center[i][0],
                slice_.GetPoint(j)[1] - Center[i][1],
                slice_.GetPoint(j)[2] - Center[i][2]
            ]
            alignPoints = rotation.apply(centered_point)
            theta_ = np.arctan2(alignPoints[1], alignPoints[0])
            new_points.append([R_map[i]*np.cos(theta_), R_map[i]*np.sin(theta_), 0])

    
    new_points_sorted = ReorderArray(arbitrary_points, VesselCenterline, new_points)
    vessel_polydata = Path2Point.points_to_vtp(new_points_sorted)
    VesselName = os.path.splitext(os.path.basename(path))[0]
    print(f"--- Converting {VesselName}")
    Vesseldir = os.path.join(InputFolder, f"Map_{VesselName}.vtp")
    WriteVTPFile(Vesseldir, BoldLine(vessel_polydata))

/var/folders/05/bygg4rdj3n7543jkngl6k6z40000gn/T/ipykernel_14047/3719622199.py:16: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  rotation, _ = R.align_vectors([[0, 0, 1]], [CL_direction])


--- Converting L_LCx_1
--- Converting No_Myo_0
--- Converting No_Myo_1
--- Converting L_LCx_0
--- Converting L_LCx_2
--- Converting L_LCx_3
--- Converting L_LAD_1
--- Converting L_LAD_0
--- Converting R_RCA_0
--- Converting L_LAD_2
--- Converting L_LAD_3
--- Converting aorta
--- Converting L_Diag2_0
--- Converting R_PDA_0
--- Converting R_PL_0
--- Converting R_PDA_1
--- Converting L_Diag1_0


In [70]:
PathFolder = os.path.join(InputFolder, "Paths/*.vtp")
import glob

import sys
import argparse

TessellationPath = os.path.abspath(f"{sys.path[0]}/..")
sys.path.append(TessellationPath)

from SimVascularTools.ConvertPathtoVTP import ConvertPath2VTP


args = argparse.Namespace()
args.InputFolder = os.path.join(InputFolder, "Paths")
Path2Point = ConvertPath2VTP(args)
Pathfiles = glob.glob(PathFolder)
rotation, _ = R.align_vectors([[0, 0, 1]], [CL_direction])

#file_ = ReadVTPFile(PathFolder)
"""CenterLinePoints = []
for i in range(centerline.GetNumberOfPoints()):
    CenterLinePoints.append(centerline.GetPoint(i))"""

for path in Path2Point.filenames:
    #file_ = ReadVTPFile(path)
    new_points = []
    vesselCLpoints, _ = Path2Point.pth_to_points(path)
    NPoints = len(vesselCLpoints)
    for p in vesselCLpoints:
        idx, myo_center_point = FindClosestPoint(Center, p)

        centered_point = [
            p[0] - myo_center_point[0],
            p[1] - myo_center_point[1],
            p[2] - myo_center_point[2]
        ]
        
        alignPoints = rotation.apply(centered_point)
        theta_ = np.arctan2(alignPoints[1], alignPoints[0])
        new_points.append([R_map[idx]*np.cos(theta_), R_map[idx]*np.sin(theta_), 0])

    vessel_polydata = Path2Point.points_to_vtp(new_points)
    VesselName = os.path.splitext(os.path.basename(path))[0]
    print(f"--- Converting {VesselName}")
    Vesseldir = os.path.join(InputFolder, f"Map_{VesselName}.vtp")
    WriteVTPFile(Vesseldir, BoldLine(vessel_polydata))

/var/folders/05/bygg4rdj3n7543jkngl6k6z40000gn/T/ipykernel_14047/3284744579.py:17: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  rotation, _ = R.align_vectors([[0, 0, 1]], [CL_direction])


--- Converting L_LCx_1
--- Converting No_Myo_0
--- Converting No_Myo_1
--- Converting L_LCx_0
--- Converting L_LCx_2
--- Converting L_LCx_3
--- Converting L_LAD_1
--- Converting L_LAD_0
--- Converting R_RCA_0
--- Converting L_LAD_2
--- Converting L_LAD_3
--- Converting aorta
--- Converting L_Diag2_0
--- Converting R_PDA_0
--- Converting R_PL_0
--- Converting R_PDA_1
--- Converting L_Diag1_0
